# 02 RAG | 01 Chat Completion Text | 07 Responses SDK

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIResponseClient

The `AzureOpenAIClient` from the Azure.AI.OpenAI NuGet package offers a unified interface to create various specialized clients, each designed to handle specific tasks.

In this notebook the `OpenAIResponseClient` is used.

The `OpenAIResponseClient` is a .NET SDK client for OpenAI’s Responses API that streamlines multi-turn conversations with text, images, streaming, and function calling—all in one flexible interface.

The Responses API unifies the straightforward interface of the Chat Completions API with the powerful function-calling features of the Assistants API.

In [6]:
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.4"
#r "nuget: DotNetEnv, 3.1.1"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI;
using OpenAI.Responses;
using DotNetEnv;
using System.IO;
using System.ClientModel;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "CHATCOMPLETION_DEPLOYMENTNAME not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);
OpenAIResponseClient responseClient = azureOpenAIClient.GetOpenAIResponseClient(chatCompletionDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"OpenAIResponseClient created...");

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.4 DotNetEnv, 3.1.1

AzureOpenAI Client created...
OpenAIResponseClient created...


## Step 1: Basic Interaction

The following cell, demonstrate a basic interaction using the `OpenAIResponseClient` object. 

The `ResponseCreationOptions` object gives more fine granular control how the model should process the provided prompt.

The `Instructions` property on the `ResponseCreationOptions` object is where you define the assistant's behavior, tone, or domain-specific guidance when you're working with Azure's Responses API.

Instructions: `"You are an AI assistant who helps finding information around international sport events. Don't answer questions that are not related to sport events."`

Prompt: `"Who won the super bowl 2024?"`

Since the GPT-4o model is trained with data only up to October 2023, it may either decline to answer or provide a speculative result about the Super Bowl 2024.

In [7]:
ResponseCreationOptions options = new ResponseCreationOptions
{
    Instructions = "You are an AI assistant who helps finding information around international sport events. Don't answer questions that are not related to sport events.",
    TruncationMode = ResponseTruncationMode.Auto
};

OpenAIResponse response = await responseClient.CreateResponseAsync(
    [ResponseItem.CreateUserMessageItem("Who won the super bowl 2024?")],
    options);

Console.WriteLine($"Model response: \n {response.GetOutputText()}"); 

Model response: 
 I'm sorry, but I don't have information on the winner of the Super Bowl 2024 as my knowledge only goes up until October 2021. You may want to check the latest sports news or the NFL's official website for the most current information.


## Step 2: Complex interaction - Chat History

### Prepare chat history

A simulated chat history is provided using a `List<ResponseItem>` with the following information. 

```html
    - User: Who won the Super Bowl in 2024?
    - Assistant: The Kansas City Chiefs won the Super Bowl 2024. The match ended with a score of 38-35.
    - User: Who did you tell me won the Super Bowl 2024? Provide just the winning team and the score.

This example demonstrates how chat history lets the assistant recall previous messages. It enables multi-turn interactions, such as responding to follow-ups based on earlier answers.

In [16]:
ResponseCreationOptions options = new ResponseCreationOptions
{
    Instructions = "You are an AI assistant who helps finding information around international sport events. Don't answer questions that are not related to sport events.",
    TruncationMode = ResponseTruncationMode.Auto
};


List<ResponseItem> messages = new List<ResponseItem>
{
    //ResponseItem.CreateDeveloperMessageItem("You are an AI assistant who helps finding information around international sport events. Don't answer questions that are not related to sport events."),
    ResponseItem.CreateUserMessageItem("Who won the Super Bowl 2024?"),
    ResponseItem.CreateAssistantMessageItem("The Kansas City Chiefs won the Super Bowl 2024. The match ended with a score of 38-35."),
    ResponseItem.CreateUserMessageItem("Who did you say won the 2024 Super Bowl? Just give me the winning team and the score. In this format: <team>, <score>."),
};

Console.WriteLine($"Chat messages created with {messages.Count} items.");

Chat messages created with 3 items.


### Model call

The chat history is provided to the model for completion.

In [17]:
OpenAIResponse response = await responseClient.CreateResponseAsync(messages, options);

Console.WriteLine($"Model response: \n {response.GetOutputText()}"); 

Model response: 
 Kansas City Chiefs, 38-35.


## Step 3: Streaming response

`CreateResponseStreamingAsync()` provides results as stream as soon as they are provided by the model.

Streaming is useful when you want faster, incremental access to a model’s output instead of waiting for the full response.

In [ ]:
//AsyncCollectionResult<StreamingResponseUpdate> responseUpdates = responseClient.CreateResponseStreamingAsync("Who won the French Open 2012?");
AsyncCollectionResult<StreamingResponseUpdate> responseUpdates = responseClient.CreateResponseStreamingAsync(messages, options);

Console.WriteLine($"Model response streaming:");
await foreach (StreamingResponseUpdate update in responseUpdates)
{
    if (update is StreamingResponseOutputTextDeltaUpdate outputTextUpdate)
    {
        Console.Write(outputTextUpdate.Delta);
    }
}

Model response streaming:
Kansas City Chiefs, 38-35.